<a href="https://colab.research.google.com/github/kar-elizabeth/EQUIPAJE/blob/main/Copia_de_060825_360_DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Leer el archivo Excel
archivo = "060825.xlsx"  # Asegúrate de que el nombre del archivo coincida exactamente
excel_data = pd.read_excel(archivo)

# Mostrar las primeras filas
excel_data.head()


,NOMBRE,APELLIDOS,CORREO_ELECTRONICO,TELEFONO_MOVIL,CODIGO_ORIGEN,ORIGEN,CODIGO_DESTINO,DESTINO,SERVICIO,FECHA_SERVICIO,...,COMODIDAD,RECOMENDACIÓN,INCONVENIENTES,COMENTARIOS_INCONVENIENTE_1,COMENTARIOS_INCONVENIENTE_2,COMENTARIO_EXPERIENCIA,CATEGORIA,AMENIDAD,DIV ORIGEN,DIV DESTINO
0,Oswaldo,Sánchez,oswaldosarro@gmail.com,5.525387e+09,PSOL,plaza del sol,MEX,norte,778,2025-01-01,...,5.0,9.0,Wifi;,Ne me pude conectar,NaN,Excelente,Positivo,Servicio,Guadalajara,México
1,Maria Ezhnevit,Segura,ezhnevits@gmail.com,4.437251e+09,MEX,norte,MORE,terminal de autobuses,750,2025-01-01,...,5.0,10.0,NaN,NaN,NaN,Gracias al personal de Primera Plus a mi llega...,Positivo,Personal,México,Morelia
2,SALVADOR,MARTINEZ,msalvador6@yahoo.com,3.512822e+09,ZAM,central de autobuses,GUAD,central nueva,870428,2025-01-01,...,4.0,8.0,Baño;,No hay agua en ellavabo,NaN,Buena,Positivo,Servicio,Irapuato,Guadalajara
3,Cesar Octavio,Alonzo Chitala,alonzesae@gmail.com,8.311669e+09,GUAD,central nueva,MEX,norte,64880,2025-01-01,...,5.0,10.0,NaN,NaN,NaN,Buena la experiencia de viajar,Positivo,Servicio,Guadalajara,México
4,Jesus,Cortez,jesus_ortega123@hotmail.com,3.741008e+09,URU,central de autobuses,GUAD,central nueva,422,2025-01-01,...,5.0,10.0,NaN,NaN,NaN,Puntualidad y comodidad,Positivo,Servicio,Morelia,Guadalajara


In [ ]:
import pandas as pd

# Leer el archivo Excel (ya debes haberlo subido con files.upload)
archivo = "060825.xlsx"
df = pd.read_excel(archivo)

# Lista de divisiones origen que vamos a separar
divisiones = ["Guadalajara", "México", "Irapuato", "Morelia", "Querétaro", "León", "Celaya"]

# Crear un archivo Excel con varias hojas
with pd.ExcelWriter("DIV_ORIGEN_SEPARADO.xlsx") as writer:
    for division in divisiones:
        df_filtrado = df[df["DIV ORIGEN"] == division]
        df_filtrado.to_excel(writer, sheet_name=division, index=False)

print("✅ Archivo DIV_ORIGEN_SEPARADO.xlsx creado con éxito.")


✅ Archivo DIV_ORIGEN_SEPARADO.xlsx creado con éxito.


In [ ]:
from google.colab import files
files.download("DIV_ORIGEN_SEPARADO.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Leer el archivo con las 7 hojas
archivo = "DIV_ORIGEN_SEPARADO.xlsx"
xls = pd.ExcelFile(archivo)
hojas = xls.sheet_names  # Lista de hojas

# Crear nuevo archivo con rankings
with pd.ExcelWriter("RANKINGS_ANALISIS.xlsx", engine="openpyxl") as writer:
    for hoja in hojas:
        df_hoja = xls.parse(hoja)

        # Caso 3: Ranking por CATEGORIA
        ranking_categoria = df_hoja["CATEGORIA"].value_counts().reset_index()
        ranking_categoria.columns = ["CATEGORIA", "TOTAL"]
        # Truncate sheet name to 31 characters if needed for openpyxl
        sheet_name_cat = f"{hoja}_Cat"[:31]
        ranking_categoria.to_excel(writer, sheet_name=sheet_name_cat, index=False)

        # Caso 4: Solo Negativo y Neutral → Ranking por CODIGO_ORIGEN
        filtro_neg_neu = df_hoja[df_hoja["CATEGORIA"].isin(["Negativo", "Neutral"])]
        ranking_codigo_origen = filtro_neg_neu["CODIGO_ORIGEN"].value_counts().reset_index()
        ranking_codigo_origen.columns = ["CODIGO_ORIGEN", "TOTAL"]
        # Truncate sheet name to 31 characters if needed for openpyxl
        sheet_name_cod = f"{hoja}_CodOrig"[:31]
        ranking_codigo_origen.to_excel(writer, sheet_name=sheet_name_cod, index=False)

        # Caso 5: Ranking por AMENIDAD según CATEGORIA (Negativo/Neutral)
        ranking_amenidad = (
            filtro_neg_neu.groupby(["CATEGORIA", "AMENIDAD"])
            .size()
            .reset_index(name="TOTAL")
            .sort_values(["CATEGORIA", "TOTAL"], ascending=[True, False])
        )
        # Truncate sheet name to 31 characters if needed for openpyxl
        sheet_name_amen = f"{hoja}_Amen"[:31]
        ranking_amenidad.to_excel(writer, sheet_name=sheet_name_amen, index=False)

print("✅ RANKINGS_ANALISIS.xlsx creado con éxito.")

SyntaxError: incomplete input (ipython-input-3952855549.py, line 1)

In [ ]:
import pandas as pd

# Leer el archivo Excel con varias hojas
archivo = "DIV_ORIGEN_SEPARADO1.xlsx"
xls = pd.ExcelFile(archivo)
hojas = xls.sheet_names

# Crear nuevo archivo con rankings por hoja y categoría
with pd.ExcelWriter("RANKING_CATEGORIA_CODIGO.xlsx", engine="openpyxl") as writer:
    for hoja in hojas:
        df_hoja = xls.parse(hoja)

        # Verificar que las columnas existan
        if "CATEGORIA" in df_hoja.columns and "CODIGO_ORIGEN" in df_hoja.columns:
            # Crear ranking por cada categoría
            categorias = ["Neutral", "Positivo", "Negativo"]
            for cat in categorias:
                df_cat = df_hoja[df_hoja["CATEGORIA"] == cat]
                ranking = df_cat["CODIGO_ORIGEN"].value_counts().reset_index()
                ranking.columns = ["CODIGO_ORIGEN", "TOTAL"]
                ranking.to_excel(writer, sheet_name=f"{hoja[:20]}_{cat}", index=False)
        else:
            print(f"⚠️  La hoja '{hoja}' no contiene las columnas necesarias.")

print("✅ Archivo RANKING_CATEGORIA_CODIGO.xlsx generado.")


✅ Archivo RANKING_CATEGORIA_CODIGO.xlsx generado.


In [ ]:
from google.colab import files
files.download("RANKING_CATEGORIA_CODIGO.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Leer el archivo Excel con varias hojas
archivo = "DIV_ORIGEN_SEPARADO1.xlsx"
xls = pd.ExcelFile(archivo)
hojas = xls.sheet_names

# Crear nuevo archivo de salida
with pd.ExcelWriter("INFORME_CATEGORIA_CODIGO.xlsx", engine="openpyxl") as writer:
    for hoja in hojas:
        df_hoja = xls.parse(hoja)

        # Verifica que las columnas necesarias existan
        if "CATEGORIA" in df_hoja.columns and "CODIGO_ORIGEN" in df_hoja.columns:
            # Agrupar por CATEGORIA y CODIGO_ORIGEN, contar registros
            resumen = (
                df_hoja.groupby(["CATEGORIA", "CODIGO_ORIGEN"])
                .size()
                .reset_index(name="TOTAL")
                .sort_values(["CATEGORIA", "TOTAL"], ascending=[True, False])
            )

            # Guardar todo el ranking en una sola hoja por ciudad
            resumen.to_excel(writer, sheet_name=hoja[:31], index=False)
        else:
            print(f"⚠️  La hoja '{hoja}' no contiene las columnas necesarias.")

print("✅ Archivo INFORME_CATEGORIA_CODIGO.xlsx creado con éxito.")


✅ Archivo INFORME_CATEGORIA_CODIGO.xlsx creado con éxito.


In [ ]:
from google.colab import files
files.download("INFORME_CATEGORIA_CODIGO.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Leer el archivo Excel con varias hojas
archivo = "DIV_ORIGEN_SEPARADO1.xlsx"
xls = pd.ExcelFile(archivo)
hojas = xls.sheet_names

# Crear nuevo archivo con datos filtrados por categoría, manteniendo toda la información original
with pd.ExcelWriter("RANKING_SUSTENTADO.xlsx", engine="openpyxl") as writer:
    for hoja in hojas:
        df_hoja = xls.parse(hoja)

        # Verificar que las columnas clave existan
        if "CATEGORIA" in df_hoja.columns and "CODIGO_ORIGEN" in df_hoja.columns:
            categorias = ["Neutral", "Positivo", "Negativo"]
            for cat in categorias:
                # Filtramos toda la información original por categoría
                df_cat = df_hoja[df_hoja["CATEGORIA"] == cat]

                # Ordenamos por frecuencia de CODIGO_ORIGEN (opcional)
                df_cat = df_cat.copy()
                df_cat["TOTAL_POR_ORIGEN"] = df_cat.groupby("CODIGO_ORIGEN")["CODIGO_ORIGEN"].transform("count")
                df_cat = df_cat.sort_values(by=["TOTAL_POR_ORIGEN", "CODIGO_ORIGEN"], ascending=[False, True])

                # Guardamos la hoja con todos los detalles
                nombre_hoja = f"{hoja[:20]}_{cat}"
                df_cat.to_excel(writer, sheet_name=nombre_hoja[:31], index=False)  # Excel solo permite 31 caracteres
        else:
            print(f"⚠️  La hoja '{hoja}' no tiene las columnas necesarias.")

print("✅ Archivo RANKING_SUSTENTADO.xlsx generado con toda la información.")


✅ Archivo RANKING_SUSTENTADO.xlsx generado con toda la información.


In [ ]:
import pandas as pd
from collections import Counter

# Cargar el archivo (ajusta el nombre del archivo si es diferente)
archivo = "RANKING SUSTENTADO.xlsx"

# Hojas que deseas analizar
hojas_ciudades = [
    "Querétaro_Negativo", "Guadalajara_Negativo", "México_Negativo",
    "Irapuato_Negativo", "Morelia_Negativo", "León_Negativo", "Celaya_Negativo"
]

# Función para procesar cada hoja
def analizar_ciudad(nombre_ciudad, df):
    resumen = []

    # Quitar filas donde no haya amenidad
    df = df.dropna(subset=['AMENIDAD'])

    # Contar ocurrencias por amenidad
    conteo_amenidades = df['AMENIDAD'].value_counts()

    for amenidad, frecuencia in conteo_amenidades.items():
        # Filtrar las filas con esa amenidad
        df_filtrado = df[df['AMENIDAD'] == amenidad]

        # Obtener la ciudad de origen más común
        ciudad_mas_frecuente = df_filtrado['CODIGO_ORIGEN'].mode().iloc[0] if not df_filtrado['CODIGO_ORIGEN'].mode().empty else 'No disponible'

        # Combinar los comentarios en una sola cadena
        comentarios = (
            df_filtrado['INCONVENIENTES'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIOS_INCONVENIENTE_1'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIOS_INCONVENIENTE_2'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIO_EXPERIENCIA'].dropna().astype(str).tolist()
        )

        comentarios_texto = " | ".join(comentarios[:5])  # Solo mostrar primeros 5 comentarios como ejemplo

        resumen.append({
            "Ciudad": nombre_ciudad.replace("_Negativo", ""),
            "Amenidad": amenidad,
            "Recurrencia": frecuencia,
            "Ciudad_origen_frecuente": ciudad_mas_frecuente,
            "Comentarios_sustento": comentarios_texto
        })

    return pd.DataFrame(resumen)

# Procesar todas las hojas
todos_los_resultados = []

for hoja in hojas_ciudades:
    df_hoja = pd.read_excel(archivo, sheet_name=hoja)
    resultados_hoja = analizar_ciudad(hoja, df_hoja)
    todos_los_resultados.append(resultados_hoja)

# Unir todos los resultados en un solo DataFrame
df_final = pd.concat(todos_los_resultados, ignore_index=True)

# Mostrar resultados
import pandas as pd
from IPython.display import display
pd.set_option('display.max_colwidth', None)
display(df_final)

# Exportar a Excel si deseas
df_final.to_excel("Ranking_Amenidades_Negativas.xlsx", index=False)


,Ciudad,Amenidad,Recurrencia,Ciudad_origen_frecuente,Comentarios_sustento
0,Querétaro,Mantenimiento,164,QRO,Contenido multimedia; | Contenido multimedia; | Asientos; | Asientos; Contenido multimedia; Mantenimiento de la unidad; | Asientos; Wifi;
1,Querétaro,Limpieza,99,QRO,Audífonos; Limpieza de la unidad; | Baño; Limpieza de la unidad; Mantenimiento de la unidad; | Baño; | Baño; | Asientos; Baño;
2,Querétaro,Servicio,95,QRO,Audífonos; Limpieza de la unidad; Personal; | Aire acondicionado; Contenido multimedia; Wifi; | Asientos; Audífonos; Personal; | Audífonos; | Operador; Puntualidad;
3,Querétaro,Puntualidad origen,84,QRO,Asientos; Puntualidad; | Wifi; | Aire acondicionado; Contenido multimedia; | Baño; Operador; Puntualidad; | Aire acondicionado; Contenido multimedia;
4,Querétaro,Asiento,71,QRO,Asientos; Mantenimiento de la unidad; Wifi; | Asientos; | Asientos; Operador; | Asientos; | Asientos;
...,...,...,...,...,...
157,Celaya,Puntualidad,2,CYA,Contenido multimedia; Puntualidad; Wifi; | Puntualidad; | Variedad y sin censura para adultos | Últimamente se han retrasado los camiones a leon | Tiempo estimado de llegada
158,Celaya,Portal web,2,CYA,Personal; Portal web; | No me quería imprimir mis boletos porque ya era tarde y no me había fijado que no me habían llegado mis boletos hasta que iba abordar el camión | No me llegaron mis boletos al correo | SU SISTEMA NO ME DEJA FACTURAR | No me llegaron mis boletos al correo
159,Celaya,Personal,2,CYA,Audífonos; Contenido multimedia; Personal; | Personal; | Repetitivo | No me quisieron cambiar un boleto y perdí mi dinero tuve que viajar en etn | Reciclados no se escucha
160,Celaya,Personal Taquilla,2,MORO,Contenido multimedia; Personal; Puntualidad; | Personal; | not working | La de la venta de boletos trata mal | your counterdesk personel was awfull


In [ ]:
import pandas as pd

# Cargar el archivo
archivo = "RANKING SUSTENTADO.xlsx"

# Hojas a analizar
hojas_ciudades = [
    "Querétaro_Negativo", "Guadalajara_Negativo", "México_Negativo",
    "Irapuato_Negativo", "Morelia_Negativo", "León_Negativo", "Celaya_Negativo"
]

# Función para procesar cada hoja
def analizar_ciudad(nombre_ciudad, df):
    resumen = []

    # Filtrar solo filas con AMENIDAD
    df = df.dropna(subset=['AMENIDAD'])

    # Ranking por AMENIDAD
    conteo_amenidades = df['AMENIDAD'].value_counts()

    for amenidad, frecuencia in conteo_amenidades.items():
        df_filtrado = df[df['AMENIDAD'] == amenidad]

        # Modo o frecuencia de columnas clave (solo si no están vacías)
        def obtener_modo(col):
            return df_filtrado[col].mode().iloc[0] if not df_filtrado[col].mode().empty else "No disponible"

        codigo_origen = obtener_modo("CODIGO_ORIGEN")
        origen = obtener_modo("ORIGEN")
        codigo_destino = obtener_modo("CODIGO_DESTINO")
        destino = obtener_modo("DESTINO")
        servicio = obtener_modo("SERVICIO")

        # Unir los primeros 5 comentarios como ejemplo de resumen de quejas
        comentarios = (
            df_filtrado['INCONVENIENTES'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIOS_INCONVENIENTE_1'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIOS_INCONVENIENTE_2'].dropna().astype(str).tolist() +
            df_filtrado['COMENTARIO_EXPERIENCIA'].dropna().astype(str).tolist()
        )

        # Tomar solo 5 comentarios distintos
        comentarios_sustento = " | ".join(comentarios[:5]) if comentarios else "Sin comentarios"

        resumen.append({
            "Ciudad": nombre_ciudad.replace("_Negativo", ""),
            "Amenidad": amenidad,
            "Recurrencia": frecuencia,
            "CODIGO_ORIGEN": codigo_origen,
            "ORIGEN": origen,
            "CODIGO_DESTINO": codigo_destino,
            "DESTINO": destino,
            "SERVICIO": servicio,
            "Comentarios_sustento": comentarios_sustento
        })

    return pd.DataFrame(resumen)

# Procesar todas las hojas
todos_los_resultados = []

for hoja in hojas_ciudades:
    df_hoja = pd.read_excel(archivo, sheet_name=hoja)
    resultados_hoja = analizar_ciudad(hoja, df_hoja)
    todos_los_resultados.append(resultados_hoja)

# Unir todos los resultados
df_final = pd.concat(todos_los_resultados, ignore_index=True)

# Mostrar DataFrame en Colab
import pandas as pd
from IPython.display import display
pd.set_option('display.max_colwidth', None)
display(df_final)

# Exportar a archivo Excel
df_final.to_excel("Ranking_Amenidades_Sustentado.xlsx", index=False)

print("✅ Archivo 'Ranking_Amenidades_Sustentado.xlsx' generado con éxito.")


/tmp/ipython-input-3538781879.py:27: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  return df_filtrado[col].mode().iloc[0] if not df_filtrado[col].mode().empty else "No disponible"
/tmp/ipython-input-3538781879.py:27: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  return df_filtrado[col].mode().iloc[0] if not df_filtrado[col].mode().empty else "No disponible"
/tmp/ipython-input-3538781879.py:27: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  return df_filtrado[col].mode().iloc[0] if not df_filtrado[col].mode().empty else "No disponible"
/tmp/ipython-input-3538781879.py:27: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  return df_filtrado[col].mode().iloc[0] if not df_filtrado[col].mode().empty else "No disponible"
/tmp/ipython-input-3538781879.py:27: UserWarning: Unable to sort modes: '<' not supported betwee

,Ciudad,Amenidad,Recurrencia,CODIGO_ORIGEN,ORIGEN,CODIGO_DESTINO,DESTINO,SERVICIO,Comentarios_sustento
0,Querétaro,Mantenimiento,164,QRO,Terminal de Autobuses,MEX,Norte,433,Contenido multimedia; | Contenido multimedia; | Asientos; | Asientos; Contenido multimedia; Mantenimiento de la unidad; | Asientos; Wifi;
1,Querétaro,Limpieza,99,QRO,terminal de autobuses,MEX,central de autobuses,157,Audífonos; Limpieza de la unidad; | Baño; Limpieza de la unidad; Mantenimiento de la unidad; | Baño; | Baño; | Asientos; Baño;
2,Querétaro,Servicio,95,QRO,terminal de autobuses,MEX,Central de Autobuses,197,Audífonos; Limpieza de la unidad; Personal; | Aire acondicionado; Contenido multimedia; Wifi; | Asientos; Audífonos; Personal; | Audífonos; | Operador; Puntualidad;
3,Querétaro,Puntualidad origen,84,QRO,Terminal de Autobuses,COAPA,central de autobuses,1,Asientos; Puntualidad; | Wifi; | Aire acondicionado; Contenido multimedia; | Baño; Operador; Puntualidad; | Aire acondicionado; Contenido multimedia;
4,Querétaro,Asiento,71,QRO,terminal de autobuses,GUAD,central nueva,1118,Asientos; Mantenimiento de la unidad; Wifi; | Asientos; | Asientos; Operador; | Asientos; | Asientos;
...,...,...,...,...,...,...,...,...,...
157,Celaya,Puntualidad,2,CYA,Central de Autobuses,ON,Central de autobuses,907,Contenido multimedia; Puntualidad; Wifi; | Puntualidad; | Variedad y sin censura para adultos | Últimamente se han retrasado los camiones a leon | Tiempo estimado de llegada
158,Celaya,Portal web,2,CYA,Central de Autobuses,MEX,Norte,851,Personal; Portal web; | No me quería imprimir mis boletos porque ya era tarde y no me había fijado que no me habían llegado mis boletos hasta que iba abordar el camión | No me llegaron mis boletos al correo | SU SISTEMA NO ME DEJA FACTURAR | No me llegaron mis boletos al correo
159,Celaya,Personal,2,CYA,Central de Autobuses,MEX,Norte,215,Audífonos; Contenido multimedia; Personal; | Personal; | Repetitivo | No me quisieron cambiar un boleto y perdí mi dinero tuve que viajar en etn | Reciclados no se escucha
160,Celaya,Personal Taquilla,2,MORO,Central de Autobuses,GTO,Central de Autobuses,335,Contenido multimedia; Personal; Puntualidad; | Personal; | not working | La de la venta de boletos trata mal | your counterdesk personel was awfull


✅ Archivo 'Ranking_Amenidades_Sustentado.xlsx' generado con éxito.
